# Shop order prediction
Import the required libraries.

In [1]:
import csv, re
import pandas as pd

## Read csv using pandas and clean data
Using pandas, read the csv containing the stock of the products and save it to data_stock dataframe, delete the unused column 'unnamed: 4' and rename the second column to "descripcion".
Note that data_stock contain all products in database. After that, separate stock from both almacenes in two different dataframes and then merge dataframes based on description (product name). After that, sum the two stocks to get the stock_total column.
Then print the first 3 rows.

In [2]:
stock = pd.read_csv("data/Products_Stocks.csv", skiprows=1)
stock.drop(['Unnamed: 4'], axis = 1, inplace = True)
stock = stock.rename(columns={ stock.columns[1]: "descripcion" })
stock_almacen = stock.loc[stock['Almacen'] == 'Almacen central']
stock_sector3 = stock.loc[stock['Almacen'] == 'GAK Sector 3']

data_stock = pd.merge(stock_sector3[['descripcion','stock']], stock_almacen[['descripcion','stock']], on='descripcion', how='outer', sort='True')
data_stock = data_stock.rename(columns={'stock_x': 'stock_physical', 'stock_y': 'stock_online'})
data_stock = data_stock.fillna(0)
data_stock['stock_total'] = data_stock['stock_physical'] + data_stock['stock_online']
data_stock[:3]

,descripcion,stock_physical,stock_online,stock_total
0,**ERROR**Calabaza cacahuete - Esencia Rural - ...,0.0,0.0,0.0
1,**ERROR**Lechuga variada- AiguaClara- 2 piezas,0.0,0.0,0.0
2,3 Kg NARANJA -AiguaClara,0.0,0.0,0.0


Repeating the same procedure, read the csv containing the online sales for the same month last year (september 2017) and save it to data_online dataframe, delete the unused column 'unnamed: 7' and rename the second column to "descripcion" and third to "cantidad".

In [3]:
data_online = pd.read_csv("data/Productos_por_proveedor_sept2017_online_store.csv", skiprows=1)
data_online.drop(['Unnamed: 7'], axis = 1, inplace = True)
data_online = data_online.rename(columns={ data_online.columns[1]: "descripcion", data_online.columns[2]: "cantidad" })
data_online[:3]

,Proveedor,descripcion,cantidad,Precio Coste,Total coste,pvp,Total pvp
0,"AGRICOLA TUPINAIRE, S.L.",Higos Frescos -FruitalpuntBio - 1 kilo,"28,91","3,75","108,39","5,85","169,08"
1,Algamar,Alga agar-agar en copos -Algamar- 50 gr,"2,00","3,13","6,26","5,55","11,10"
2,Algamar,Alga kombu-Algamar- 100 gr,"1,00","3,57","3,57","5,40","5,40"


Same here, this case reading the csv containing the physical sales through POS for the same month last year (september 2017) and save it to data_physical dataframe.

In [4]:
data_physical = pd.read_csv("data/Productos_por_proveedor_sept2017_physical_store.csv", skiprows=1)
data_physical.drop(['Unnamed: 7'], axis = 1, inplace = True)
data_physical[:3]

,Proveedor,descripcion,cantidad,Precio coste,Total coste,pvp,total pvp
0,"AGRICOLA TUPINAIRE, S.L.",Higos Frescos -FruitalpuntBio - 1 kilo,"10,05","3,75","37,69","5,85","58,80"
1,NaN,NaN,NaN,NaN,"Total:37,69",NaN,"Total:58,80"
2,Algamar,Alga Dulse - Algamar - 100gr,"1,00","5,76","5,76","9,70","9,70"


Then we extract the columns that we need for our analysis, which are 'Proveedor', 'descripcion' and 'cantidad'

In [5]:
data_physical = data_physical[['Proveedor','descripcion','cantidad']]
data_online = data_online[['Proveedor','descripcion','cantidad']]

## Selecting data of a certain provider
Here we select the rows in dataframe which column 'Proveedor' matches 'Algamar' provider. We do it for data_physical and data_online dataframes

In [11]:
data_provider_physical = data_physical.loc[data_physical['Proveedor'] == "Algamar"]
data_provider_online = data_online.loc[data_online['Proveedor'] == "Algamar"]

First we merge data_physical and data_online tables by merging them based on the 'descripcion' column.

In [12]:
items_in_common = pd.merge(data_provider_physical[['descripcion','cantidad']], data_provider_online[['descripcion','cantidad']], on=['descripcion'], how = 'outer')
items_in_common = items_in_common.rename(columns={'cantidad_x': 'sales_physical', 'cantidad_y': 'sales_online'})
items_in_common = items_in_common.fillna(0)
items_in_common['sales_online'] = items_in_common['sales_online'].str.replace(',','.')
items_in_common['sales_physical'] = items_in_common['sales_physical'].str.replace(',','.')
items_in_common['sales_online'] = items_in_common['sales_online'].astype('float64') 
items_in_common['sales_physical'] = items_in_common['sales_physical'].astype('float64') 
items_in_common = items_in_common.fillna(0)
items_in_common['sales_total'] = items_in_common['sales_physical'] + items_in_common['sales_online']
items_in_common


,descripcion,sales_physical,sales_online,sales_total
0,Alga Dulse - Algamar - 100gr,1.0,0.0,1.0
1,Arroz- lentejas pardinas y algas - Algamar- 50...,1.0,2.0,3.0
2,Espagueti tierra y mar (pasta con algas) - Alg...,2.0,0.0,2.0
3,Flores de Mar (pasta integral con algas)- Alga...,2.0,2.0,4.0
4,Pate de algas - Algamar- 200 gr,1.0,1.0,2.0
5,Pisto con algas - Algamar- 420 gr,1.0,0.0,1.0
6,Quinoa con Algas - Algamar - 500g,1.0,0.0,1.0
7,Sopa de arroz- verduras y algas - Algamar- 50...,1.0,0.0,1.0
8,Sopa juliana con algas - Algamar- 150 gr,1.0,3.0,4.0
9,Tortitas 5 cereales con algas - Algamar - 140gr,1.0,0.0,1.0


In [13]:
output = pd.merge(items_in_common, data_stock, on=['descripcion'], how = 'inner') #non existent products had 0 sales 
output['to_buy'] = output['sales_total'] - output['stock_total']
output

,descripcion,sales_physical,sales_online,sales_total,stock_physical,stock_online,stock_total,to_buy
0,Alga Dulse - Algamar - 100gr,1.0,0.0,1.0,0.0,2.0,2.0,-1.0
1,Arroz- lentejas pardinas y algas - Algamar- 50...,1.0,2.0,3.0,0.0,5.0,5.0,-2.0
2,Espagueti tierra y mar (pasta con algas) - Alg...,2.0,0.0,2.0,0.0,10.0,10.0,-8.0
3,Flores de Mar (pasta integral con algas)- Alga...,2.0,2.0,4.0,0.0,8.0,8.0,-4.0
4,Pate de algas - Algamar- 200 gr,1.0,1.0,2.0,0.0,-3.0,-3.0,5.0
5,Pisto con algas - Algamar- 420 gr,1.0,0.0,1.0,0.0,-3.0,-3.0,4.0
6,Quinoa con Algas - Algamar - 500g,1.0,0.0,1.0,0.0,1.0,1.0,0.0
7,Sopa de arroz- verduras y algas - Algamar- 50...,1.0,0.0,1.0,0.0,3.0,3.0,-2.0
8,Sopa juliana con algas - Algamar- 150 gr,1.0,3.0,4.0,0.0,-3.0,-3.0,7.0
9,Tortitas 5 cereales con algas - Algamar - 140gr,1.0,0.0,1.0,0.0,4.0,4.0,-3.0
